# Производительность модели ключ-значение на примере работы с Riak

<p> https://en.wikipedia.org/wiki/Riak
<p> http://riak-python-client.readthedocs.io/en/2.7.0/

```json
kv = {
    'key': [
        "reg_id": <string>,
        "reg_name": <string>,
        "year": <number>,
        "group_id": <number>,
        "group_name": <string>,
        "category": <string>,
        "gender": <string>,
        "value": <number>,
    ]
}
```

In [1]:
import riak

In [2]:
client = riak.RiakClient(pb_port=8087)
bucket = client.bucket('crimestatsocial')

## Получаем данные по ключу

In [3]:
bucket.get('1').data

{'reg_id': '1100',
 'reg_name': 'Российская Федерация',
 'year': 2016,
 'group_id': 14002,
 'group_name': 'по возрасту',
 'category': '14-15',
 'gender': 'Мужчины',
 'value': 13573}

In [4]:
%timeit bucket.get('1').data

4.62 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [5]:
comm = """
    function (v) {
        var parse_data = JSON.parse(v.values[0].data);
        if(parse_data["year"] == 2016)
            result = parse_data;
        else
            result = {};
        return [result];
    }"""
client \
    .map(comm) \
    .add('crimestatsocial', '1') \
    .add('crimestatsocial', '2') \
    .run()

[{'reg_id': '1100',
  'reg_name': 'Российская Федерация',
  'year': 2016,
  'group_id': 14002,
  'group_name': 'по возрасту',
  'category': '16-17',
  'gender': 'Мужчины',
  'value': 29447},
 {'reg_id': '1100',
  'reg_name': 'Российская Федерация',
  'year': 2016,
  'group_id': 14002,
  'group_name': 'по возрасту',
  'category': '14-15',
  'gender': 'Мужчины',
  'value': 13573}]

In [6]:
comm = """
    function (v) {
        var parse_data = JSON.parse(v.values[0].data);
        if(parse_data["year"] == 2016)
            result = 1;
        else
            result = 0;
        return [result];
    }"""
mr = client.map(comm).reduce_sum()
for i in range(100):
    mr.add('crimestatsocial', str(i))
mr.run()

[24]

In [7]:
client.close()